# Introduction
This Jupyter Notebook aims to analyze user behavior across different lending protocols, focusing on how users interact with various tokens as collateral and debt. Specifically, we will investigate the looping behavior of users, where assets are borrowed on one protocol and then deposited as collateral in another protocol. This analysis will help us understand the extent and impact of such behaviors on the lending ecosystem.

# Objectives
### Load the Data

- We will load loan data for multiple lending protocols from Google Cloud Storage. The datasets contain detailed information about users, their collateral, and debt across different protocols.
- The data loading process will be implemented flexibly to allow easy switching between data sources (e.g., from cloud storage to a local database).

### Visualize User Behavior
- We will create visualizations to track the behavior of individual users across lending protocols, focusing on specific tokens such as "ETH", "wBTC", "USDC", "DAI", "USDT", "wstETH", "LORDS", "STRK", "UNO", and "ZEND".
- The visualizations will help answer several key questions:
  - How many users have borrowed an asset on one protocol and deposited the asset as collateral in another protocol?
  - How many users have completed a loop, i.e., deposited token X as collateral, borrowed token Y, deposited Y in another protocol, and borrowed X again?
  - What is the total dollar amount of tokens involved in these loops? How much are the deposits multiplied?
  - Which protocols are most subject to looping behavior? How do they compare on a per-token basis?

# Analysis and Insights
The analysis will not only address the predefined questions but also explore additional metrics and hypotheses that may arise during the investigation.
Meaningful outputs and insights will be provided, documenting the findings and their implications for the lending protocols.

# Loading the data

### From local Database

#### Postgres

In [ ]:
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display  # Only needed for Jupyter Notebook

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host='your_host',
    user='your_username',
    password='your_password',
    dbname='loans_db'
)

# List of protocols (table names in the PostgreSQL database)
protocols = ["zklend", "nostra_alpha", "nostra_mainnet", "hashstack_v0", "hashstack_v1"]

for protocol in protocols:
    print(f"Processing {protocol}...")
    
    # Query the data from the PostgreSQL database
    query = f"SELECT * FROM {protocol}"
    df = pd.read_sql_query(query, conn)
    
    pd.set_option('display.max_columns', None)  # Display all columns
    pd.set_option('display.max_colwidth', None)  # Display full column width
    pd.set_option('display.width', None)  # Adjust display width

    # Display the first rows
    display(df.head())

# Close the connection
conn.close()


#### MySQL

In [ ]:
import pandas as pd
import mysql.connector
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display  # Only needed for Jupyter Notebook

# Connect to the MySQL database
conn = mysql.connector.connect(
    host='your_host',
    user='your_username',
    password='your_password',
    database='loans_db'
)

# List of protocols (table names in the MySQL database)
protocols = ["zklend", "nostra_alpha", "nostra_mainnet", "hashstack_v0", "hashstack_v1"]

for protocol in protocols:
    print(f"Processing {protocol}...")
    
    # Query the data from the MySQL database
    query = f"SELECT * FROM {protocol}"
    df = pd.read_sql_query(query, conn)
    
    pd.set_option('display.max_columns', None)  # Display all columns
    pd.set_option('display.max_colwidth', None)  # Display full column width
    pd.set_option('display.width', None)  # Adjust display width

    # Display the first rows
    display(df.head())

# Close the connection
conn.close()


### From GCS

In [15]:
import pandas as pd
import pyarrow.parquet as pq
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from io import BytesIO

# URLs of the loans files for all lending protocols
parquet_urls = {
    "zklend": "https://storage.googleapis.com/derisk-persistent-state/zklend_data/loans.parquet",
    "nostra_alpha": "https://storage.googleapis.com/derisk-persistent-state/nostra_alpha_data/loans.parquet",
    "nostra_mainnet": "https://storage.googleapis.com/derisk-persistent-state/nostra_mainnet_data/loans.parquet",
    "hashstack_v0": "https://storage.googleapis.com/derisk-persistent-state/hashstack_v0_data/loans.parquet",
    "hashstack_v1": "https://storage.googleapis.com/derisk-persistent-state/hashstack_v1_data/loans.parquet"
}

dataframes = {}

for protocol,url in parquet_urls.items():
    # Download the file
    response = requests.get(url)
    response.raise_for_status()  # Ensure the request was successful

    # Read the Parquet file into a Pandas DataFrame
    with BytesIO(response.content) as f:
        table = pq.read_table(f)
        df = table.to_pandas()
    
    # Store dataframe in dictionary
    dataframes[protocol] = df

    pd.set_option('display.max_columns', None)  # Display all columns
    pd.set_option('display.max_colwidth', None)  # Display full column width
    pd.set_option('display.width', None)  # Adjust display width


    # Display the first rows
    display(df.head())



,User,Protocol,Collateral (USD),Risk-adjusted collateral (USD),Debt (USD),Health factor,Standardized health factor,Collateral,Debt
0,0x4306021e30f9577351207140f90425b3e9e102ec5a424843d699cd6c9c05307,zkLend,5744.568231,4289.009524,22.162648,193.524234,193.524234,"USDC: 113.3876, USDT: 4610.7524, STRK: 904.5577","USDC: 10.0284, USDT: 10.0302, wstETH: 0.0006"
1,0x30b399e06903676ada3eccd5522e0cca4c4ad0101468c0ac407a56aa1a0ed3c,zkLend,37.671463,30.137170,0.000000,inf,inf,ETH: 0.0126,
2,0x2f006034f567d5c2431bc4104b2cc7a1bf8f004bd00cbbfbf7b656e6e6c443c,zkLend,102.450086,81.960069,0.387499,211.510582,211.510582,"ETH: 0.0311, USDC: 6.5088, USDT: 3.0144",ETH: 0.0005
3,0x43e9ee859c0f85a6d5ab3f7ad26c50b9e9d8a8e10d0d71a3f4200192c6871a2,zkLend,-5.156963,-4.125570,0.000000,inf,inf,,
4,0x22dd5ed1e4d359eca2e772ecefa57e31bb7756772850081dc12217dd03cffbc,zkLend,213.311298,157.651127,0.000000,inf,inf,"wBTC: 0.0018, DAI: 23.1396, USDT: 83.3628",


,User,Protocol,Collateral (USD),Risk-adjusted collateral (USD),Debt (USD),Health factor,Standardized health factor,Collateral,Debt
0,0x6810f5565965ac915feee7d57d3225cd470ce54a40b5b53d69d9f4ed59bd719,Nostra Alpha,5.012320,4.511088,0.000000,inf,inf,USDC: 5.0136,
1,0x1b706b18846667c44fdf2dc302ed90b98c0ae3aa6e041fdbdea1289d0284b03,Nostra Alpha,32.454788,26.051526,0.000000,inf,inf,"ETH: 0.0070, USDC: 0.8772, USDT: 10.6526",
2,0x3def891ad08c209a1fcc67607c55b32e0d18137b00f90bd393b4b85867ad435,Nostra Alpha,14.938994,11.951195,0.000000,inf,inf,ETH: 0.0050,
3,0x42dbc9170cd78224c4d013fd01f2401da0e66f9bba4bb6505224ba93e27a2bb,Nostra Alpha,4.009317,3.407946,1.113672,2.907094,2.907094,"USDC: 2.0055, USDT: 2.0051",USDT: 1.1141
4,0x2d50b82f53cc24f44d562207009f89f60b04ba98bff6deba999b50b964c9dff,Nostra Alpha,4.139529,3.725576,0.498447,5.979490,5.979490,USDC: 4.1406,wBTC: 0.0000


,User,Protocol,Collateral (USD),Risk-adjusted collateral (USD),Debt (USD),Health factor,Standardized health factor,Collateral,Debt
0,0x5295882956890bb9c05e54c1e7efa903a4a21083ebd6a3c175507f74e549574,Nostra Mainnet,1702.601958,1362.208392,0.000000,inf,inf,"ETH: 0.5515, USDC: 2.0574, DAI: 12.5399, USDT: 5.2209, wstETH: 0.0101, STRK: 0.3490",
1,0x2d50b82f53cc24f44d562207009f89f60b04ba98bff6deba999b50b964c9dfe,Nostra Mainnet,0.103021,0.092719,0.000000,inf,inf,USDC: 0.1030,
2,0x1b706b18846667c44fdf2dc302ed90b98c0ae3aa6e041fdbdea1289d0284b03,Nostra Mainnet,1453.043720,1162.434976,0.000000,inf,inf,ETH: 0.4865,
3,0x16e26f25bc7940de9c75347dab436f733c25e6da6e492a1eb74c218cd7d05ae,Nostra Mainnet,10329.173299,8479.192061,1368.895753,5.884475,5.884475,"ETH: 2.5902, USDC: 2159.7961, USDT: 433.6514, STRK: 0.3070",USDT: 1369.3983
4,0x582b266f28c77fc6a37490e3ba13b97406c5d72294939937c423d35c5793533,Nostra Mainnet,0.000000,0.000000,0.000000,inf,inf,,


,User,Protocol,Collateral (USD),Risk-adjusted collateral (USD),Debt (USD),Health factor,Standardized health factor,Collateral,Debt
0,0x5b55db55f5884856860e63f3595b2ec6b2c9555f3f507b4ca728d8e427b7864,Hashstack V0,0.000000,0.000000,0.0,inf,inf,,
1,0x5b55db55f5884856860e63f3595b2ec6b2c9555f3f507b4ca728d8e427b7864,Hashstack V0,14.389145,14.389145,0.0,inf,inf,USDC: 14.3929,
2,0x5b55db55f5884856860e63f3595b2ec6b2c9555f3f507b4ca728d8e427b7864,Hashstack V0,0.000000,0.000000,0.0,inf,inf,,
3,0x1e307316999c4830ada67b495d03246ab205e77853bb117da7b555122a33bc7,Hashstack V0,0.000000,0.000000,0.0,inf,inf,,
4,0x74061d07a0fbd8ffc376dc4f593c69854c51b6c2fe5596d88452e9efdd76864,Hashstack V0,0.000000,0.000000,0.0,inf,inf,,


,User,Protocol,Collateral (USD),Risk-adjusted collateral (USD),Debt (USD),Health factor,Standardized health factor,Collateral,Debt
0,0x251e01d33f75076732f5c7c671d1e6918f3cd0f36a4dedcc35e642297f30d2a,Hashstack V1,12.098501,12.098501,4.498348,2.689543,2.586099,"USDT: 5.0000, JediSwap: WBTC/USDC Pool: 0.0000",USDT: 4.5000
1,0x251e01d33f75076732f5c7c671d1e6918f3cd0f36a4dedcc35e642297f30d2a,Hashstack V1,10.341679,10.341679,7.168104,1.442736,1.387246,"USDT: 3.9997, mySwap: WBTC/USDC Pool: 0.0099",ETH: 0.0024
2,0x14afb3508a365a406cc5dd1578d987a3b15d53df661a1dbd07049b57199e4d1,Hashstack V1,0.000000,0.000000,0.000000,inf,inf,,
3,0x251e01d33f75076732f5c7c671d1e6918f3cd0f36a4dedcc35e642297f30d2a,Hashstack V1,5.998194,5.998194,3.998948,1.499943,1.442253,"USDC: 4.0000, USDT: 2.0000",USDC: 4.0000
4,0x14afb3508a365a406cc5dd1578d987a3b15d53df661a1dbd07049b57199e4d1,Hashstack V1,0.000000,0.000000,0.000000,inf,inf,,
